**SA463A &#x25aa; Data Wrangling and Visualization &#x25aa; Fall 2020 &#x25aa; Foraker and Uhan**

# Lesson 10. Cartographic Visualization

## In this lesson...

- **Cartography** &mdash; the study and practice of map-making &mdash; has a rich history spanning centuries of discovery and design


- **Cartographic visualization** leverages mapping techniques to convey data containing spatial information, such as locations, routes, or trajectories on the surface of the Earth


- In this lesson, we'll learn about the basics of creating maps and visualizing spatial data with Altair

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Setup

- Let's start by importing our good friends, Pandas and Altair:

In [1]:
import pandas as pd
import altair as alt

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Preliminaries: more ways of specifying data in Altair

- So far in this course, we have specified data in a Chart object as a *DataFrame*


- There are a few other useful ways to specify data

### URL strings

- We can give a Chart object a **URL string** pointing to a CSV or JSON file, either on the local file system or in the cloud


- For example, the Seattle/New York weather dataset we used in previous lessons lives in the cloud here: 

        https://vega.github.io/vega-datasets/data/weather.csv
    
    
- We can pass this URL directly to a Chart object like this:    

In [2]:
weather_url = 'https://vega.github.io/vega-datasets/data/weather.csv'

alt.Chart(weather_url).mark_point().encode(
    alt.X('temp_min:Q'),
    alt.Y('temp_max:Q')
)

alt.Chart(...)

### UrlData objects

- When we use a URL string, Altair automatically infers whether the variables in the dataset are numbers, strings, dates, or Booleans (True/False)


- Unfortunately, Altair doesn't always get this right


- We can help Altair by specifying a URL with `alt.UrlData()`


- In the same folder as this notebook, there is a CSV file `data/airports.csv` containing a dataset of airports in the United States, partly based on [the datasets found here](http://ourairports.com/data/)


- The CSV file contains the following columns:

| Column | Description |
| :- | :- |
| `iata` | Airport code |
| `name` | Name of airport | 
| `city` | City of airport |
| `state` | State of airport |
| `latitude` | Latitude of airport |
| `longitude` | Longitude of airport |
| `routes` | Number of routes originating from airport |


- Consider the following chart:

In [3]:
airports_url = 'data/airports.csv'

alt.Chart(airports_url).transform_filter(
    'datum.iata == "DFW" | datum.iata == "DCA" | datum.iata == "LAX"'
).mark_bar().encode(
    alt.Y('iata:N', 
          sort=alt.EncodingSortField(field='routes')),
    alt.X('routes:Q'),
    alt.Tooltip('routes:Q')
)

alt.Chart(...)

- We told Altair to sort the y-axis according to the number of routes, but we're not getting what we expect


- It turns out that Altair thinks the variable `routes` contains *strings*, not numbers
    - Strings are sorted "dictionary-style", based on the characters or digits, from left to right


- In the code cell below, let's specify `data/airports.csv` using `alt.UrlData()` and tell Altair how to parse the `routes` variable:

In [4]:
# Solution
airports_data = alt.UrlData(
    url='data/airports.csv',
    format=alt.DataFormat(type='csv', parse={'routes': 'number'})
)

alt.Chart(airports_data).transform_filter(
    'datum.iata == "DFW" | datum.iata == "DCA" | datum.iata == "LAX"'
).mark_bar().encode(
    alt.Y('iata:N', 
          sort=alt.EncodingSortField(field='routes')),
    alt.X('routes:Q'),
    alt.Tooltip('routes:Q')
)

alt.Chart(...)

- `alt.UrlData()` takes the following keyword arguments:
    - `url=...` specifies the URL of the dataset
    - `format=...` takes an `alt.DataFormat` object


- `alt.DataFormat()` takes the following keyword arguments:
    - `type=...` can be `'csv'`, `'json'`, `'tsv'`, `'dsv'`, or `'topojson'`
    - `parse=...` takes a dictionary that maps a variable name to one of the following variable types: `'number'`, `'string'`, `'boolean'`, or `'date'`

### Side note: Why use a URL instead of a DataFrame?

- When we give a Chart object a DataFrame, it embeds the entire dataset within the Chart object
    - This can lead to very large charts (in terms of memory/storage) and in turn, large notebooks


- Altair tries to prevent us from doing this: by default, if you try to create a Chart with a DataFrame that has more than 5000 rows, you will get an error


- On the other hand, when we point a Chart object to a CSV (or JSON) file with a URL, it streams the data directly from the file

### GeoJSON features

- Altair can also use certain types of geographic data &mdash; in particular, those represented as *GeoJSON features*


- We'll see how these work next

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## A quick example: the GeoJSON format and geoshape marks

- Up to this point, we have worked with datasets in CSV files that correspond to tabular data: rows (observations or records) and columns (variables or fields)


- **Geographic data**, such as *geographic regions* (e.g., countries, states) and *trajectories* (e.g., flight paths, subway lines) usually come in different formats


- [GeoJSON](https://geojson.org/) is a format for encoding a variety of geographic data structures with JSON files


- A **GeoJSON feature** represents a spatially bounded thing (like a country)


- To illustrate, let's look at `data/maryland.geojson`, which contains a GeoJSON feature for the boundary of Maryland, located in the same folder as this notebook


- As you can see, the `feature` contains:
    - `properties`, like the `name` of the feature
    - a `geometry` that consists a list of `coordinates` given by `[longitude, latitude]` coordinates that define the state boundary


- To visualize geographic data, Altair provides the `geoshape` mark type


- For example, we can create a Chart object based on the GeoJSON feature specified `data/maryland.geojson`, and apply the `mark_geoshape()` method, like this:

In [5]:
# Solution
alt.Chart('data/maryland.geojson').mark_geoshape()

alt.Chart(...)

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## TopoJSON

- [TopoJSON](https://github.com/topojson/topojson) is an extension of GeoJSON 


- A TopoJSON file can specify multiple GeoJSON features compactly by encoding the *topology* of geographic regions
    - In particular, encoding borders shared between features only once


- We can view TopoJSON data in a similar fashion to GeoJSON files, but with one additional step

- To illustrate, we're going to use a dataset from the `vega_datasets` Python package
    - This package provides access to a variety of [datasets hosted in the cloud by the Vega team](https://github.com/vega/vega-datasets)


- Let's import the `data` object `vega_datasets`:

In [6]:
from vega_datasets import data

- The `data` object contains pointers to a number of datasets


- For example, let's look at `data.world_110m`, which contains the boundaries of the world countries at a 110 meter resolution


- We can get the URL of this dataset like this:

In [7]:
world_url = data.world_110m.url
world_url

'https://vega.github.io/vega-datasets/data/world-110m.json'

- We can also read this file as a Python *dictionary* using `data.world_110m` as a *method*, like this:

In [8]:
world = data.world_110m()
world

{'type': 'Topology',
 'transform': {'scale': [0.0036000360003600037, 0.0016925586033320111],
  'translate': [-180, -85.60903777459777]},
 'objects': {'land': {'type': 'MultiPolygon',
   'arcs': [[[0]],
    [[1]],
    [[2]],
    [[3]],
    [[4]],
    [[5]],
    [[6]],
    [[7, 8, 9]],
    [[10, 11]],
    [[12]],
    [[13]],
    [[14]],
    [[15]],
    [[16]],
    [[17]],
    [[18]],
    [[19]],
    [[20]],
    [[21]],
    [[22]],
    [[23]],
    [[24]],
    [[25]],
    [[26]],
    [[27]],
    [[28]],
    [[29, 30]],
    [[31]],
    [[32]],
    [[33]],
    [[34]],
    [[35]],
    [[36]],
    [[37]],
    [[38]],
    [[39]],
    [[40]],
    [[41]],
    [[42, 43]],
    [[44]],
    [[45]],
    [[46]],
    [[47, 48, 49, 50]],
    [[51]],
    [[52]],
    [[53]],
    [[54]],
    [[55]],
    [[56]],
    [[57]],
    [[58]],
    [[59]],
    [[60]],
    [[61]],
    [[62, 63]],
    [[64]],
    [[65]],
    [[66]],
    [[67]],
    [[68]],
    [[69]],
    [[70]],
    [[71]],
    [[72]],
    [[73]],
   

- As a result, we can, at a high level, quickly see the `'objects'` of `data.world_110m`:

In [9]:
world['objects'].keys()

dict_keys(['land', 'countries'])

- Roughly speaking, each `object` in a TopoJSON file corresponds to a collection of GeoJSON features


- To use a TopoJSON file with a Chart object, we need to use a helper function, `alt.topo_feature()`, that extracts all the GeoJSON features from a particular TopoJSON `object`


- For example, we can visualize the `land` object in `data.world_110m` like this:

In [10]:
# Solution
alt.Chart(
    alt.topo_feature(world_url, 'land')
).mark_geoshape()

alt.Chart(...)

- Note that `alt.topo_feature()` takes the URL as the first argument, not the dataset itself


- Similarly, for the `countries` object:

In [11]:
# Solution
alt.Chart(
    alt.topo_feature(world_url, 'countries')
).mark_geoshape()

alt.Chart(...)

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Geoshape mark properties and projections

- We can customize the fill color, stroke color, and stroke widths using standard mark properties:

In [12]:
# Solution
alt.Chart(
    alt.topo_feature(world_url, 'countries')
).mark_geoshape(
    fill='black',
    stroke='white',
    strokeWidth=0.5
)

alt.Chart(...)

- We can also change the default projection of the map using the `project()` method, like this:

In [13]:
# Solution
alt.Chart(
    alt.topo_feature(world_url, 'countries')
).mark_geoshape(
    fill='black',
    stroke='white',
    strokeWidth=0.5
).project(
    type='orthographic'
)

alt.Chart(...)

- The default projection `type` is `'mercator'`


- There is a wide assortment of available projections
    - [Vega documentation on the available cartographic projections](https://vega.github.io/vega/docs/projections/#types)


- You can also use the `project()` method to zoom into a particular part of the map with the following keyword arguments:
    - `scale=...` specifies the magnification level as a number
    - `translate=...` takes a 2-element list `[tx, ty]`, which specifies the pixel coordinates of the projection's center
    

- You'll need to play around a bit with these keyword arguments to zoom into the part of the map you want


- We can focus on Europe in the `'mercator'` projection like this:

In [14]:
# Solution
alt.Chart(
    alt.topo_feature(world_url, 'countries')
).mark_geoshape(
    fill='black',
    stroke='white',
    strokeWidth=0.5
).project(
    type='mercator',
    scale=400,
    translate=[100, 550]
)

alt.Chart(...)

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Point and symbol maps

- Many tabular datasets include geographic information such as:
    - longitude and latitude coordinates
    - references to geographic regions that can be mapped to coordinates (e.g. country names, postal codes)
        
        
- We can combine this data with the *geometric* data in a GeoJSON or TopoJSON file to create maps with points or symbols

- Earlier in this lesson, we used the CSV file `data/airports.csv` that contains a dataset of airports in the United States


- Let's create a map that shows the location of each airport in this dataset


- First, let's create a base layer containing a map of the US, including the state boundaries, using the `data.us_10m` dataset from `vega_datasets`


- We start by getting the URL of this dataset, like this:

In [15]:
us_url = data.us_10m.url
us_url

'https://vega.github.io/vega-datasets/data/us-10m.json'

- Let's also read in the actual TopoJSON file as a dictionary, so we can peek at the structure and determine the objects available to us:

In [16]:
us = data.us_10m()
us['objects'].keys()

dict_keys(['counties', 'states', 'land'])

- There is a `states` object! That seems promising &mdash; let's try it out
  
    
- Let's configure our map as follows: 
    - fill color of `'lightgray'`
    - stroke color of `'white'`
    - stroke width of `1`
    - 'albersUSA'` projection
    
    
- As the name suggests, the `'albersUSA'` projection is well-suited for visualizations involving the US


- So, we can create our base layer map like this:

In [17]:
# Solution
base = alt.Chart(
    alt.topo_feature(us_url, 'states')
).mark_geoshape(
    fill='lightgray', stroke='white', strokeWidth=1
).project(
    type='albersUsa'
)

base

alt.Chart(...)

- Next, we'll create a layer of points corresponding to the locations of the airports


- Like we did earlier, let's read `data/airports.csv` using `alt.UrlData()`, making sure to specify that the `routes` variable contains numeric data

In [18]:
airports_data = alt.UrlData(
    url='data/airports.csv',
    format=alt.DataFormat(type='csv', parse={'routes': 'number'})
)

- We'll represent each observation/airport by a circle using `mark_circle()`
    - Let's make the points smaller than the default size, say `size=8`


- To put the points in the correct places, we'll use the `alt.Longitude()` and `alt.Latitude()` encoding channels
    - Note that we *don't* simply use the `alt.X()` and `alt.Y()` channels
    - There is no guarantee that `longitude` → `x` and `latitude` → `y`!
    
    
- Finally, let's add a tooltip that displays the corresponding airport code when the user mouses over each circle
    
    
- So, something like this:

In [19]:
# Solution
points = alt.Chart(airports_data).mark_circle(
    size=8
).encode(
    alt.Latitude('latitude:Q'),
    alt.Longitude('longitude:Q'),
    alt.Tooltip('iata:N')
).project(
    type='albersUsa'
)

points

alt.Chart(...)

- Now, let's layer the two charts on top of each other, make our chart a little larger, and remove the rectangular border:

In [20]:
# Solution
(base + points).properties(
    width=600,
    height=400
).configure_view(
    strokeWidth=0
)

alt.LayerChart(...)

- It would be nice to see the relative traffic of the airports &mdash; this way, we can identify major hubs


- Let's modify the map we created above to visualize the number of routes with the size encoding channel:

In [ ]:
base = alt.Chart(
    alt.topo_feature(us_url, 'states')
).mark_geoshape(
    fill='lightgray', stroke='white', strokeWidth=1
).project(
    type='albersUsa'
)

points = alt.Chart(airports_data).mark_circle(
    size=8
).encode(
    alt.Latitude('latitude:Q'),
    alt.Longitude('longitude:Q'),
    alt.Tooltip('iata:N')
).project(
    type='albersUsa'
)

(base + points).properties(
    width=600,
    height=400
).configure_view(
    strokeWidth=0
)

In [21]:
# Solution
base = alt.Chart(
    alt.topo_feature(us_url, 'states')
).mark_geoshape(
    fill='lightgray', stroke='white', strokeWidth=1
).project(
    type='albersUsa'
)

points = alt.Chart(airports_data).mark_circle(
    size=8
).encode(
    alt.Latitude('latitude:Q'),
    alt.Longitude('longitude:Q'),
    [alt.Tooltip('iata:N'), alt.Tooltip('routes:Q')],
    alt.Size('routes:Q'),
    alt.Order('routes:Q', sort='descending')
).project(
    type='albersUsa'
)

(base + points).properties(
    width=600,
    height=400
).configure_view(
    strokeWidth=0
)

alt.LayerChart(...)

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Choropleth maps and the lookup transform

- A **choropleth map** uses shaded or textured regions to visualize data values


- Sized symbol maps, like the one we just created above, are often more accurate to read
    - People tend to be better at estimating proportional differences between the area of circles than between color shades


- Nevertheless, choropleth maps are useful when too many symbols become perceptually overwhelming

- To illustrate, let's build a choropleth map of the unemployment rate per county, back in the recession year of 2008


- We'll base our visualization on two datasets, both from `vega_datasets`: 
    1. a *tab-separated values* (TSV) file that contains unemployment statistics (`data.unemployment`)
    2. a TopoJSON file that includes county boundary features (`data.usa_10m`)


- Let's store the URL of the unemployment data:

In [22]:
unemp_url = data.unemployment.url
unemp_url

'https://vega.github.io/vega-datasets/data/unemployment.tsv'

- We can also peek at the structure of this dataset by downloading it in our browser


- We see there are two columns:

| Column | Description |
| :- | :- |
| `id` | [County FIPS code](https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county) |
| `rate` | Unemployment rate |

- Earlier, we stored 
    - the URL of `data.usa_10m` in the variable `us_url`
    - the dataset as a dictionary in the variable `us`


- Digging into the dictionary `us`, we saw that one of the objects in this TopoJSON dataset is `counties`

In [23]:
us['objects']['counties'].keys()

dict_keys(['type', 'geometries'])

- Digging a bit deeper into the `counties` object, we see  that it contains a list of `geometries`, each of which has an `id`

In [24]:
us['objects']['counties']['geometries']

[{'type': 'MultiPolygon', 'arcs': [], 'id': 22051},
 {'type': None, 'id': 23023},
 {'type': None, 'id': 37031},
 {'type': None, 'id': 42045},
 {'type': 'Polygon',
  'arcs': [[0,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20,
    21,
    22,
    23,
    24,
    25],
   [26],
   [27],
   [28],
   [29],
   [30],
   [31],
   [32],
   [33],
   [34],
   [35],
   [36],
   [37],
   [38],
   [39],
   [40],
   [41],
   [42],
   [43],
   [44],
   [45],
   [46],
   [47],
   [48],
   [49],
   [50],
   [51],
   [52],
   [53],
   [54],
   [55],
   [56],
   [57],
   [58],
   [59],
   [60],
   [61],
   [62],
   [63],
   [64],
   [65],
   [66],
   [67],
   [68],
   [69],
   [70],
   [71],
   [72],
   [73],
   [74],
   [75],
   [76],
   [77],
   [78],
   [79],
   [80],
   [81],
   [82],
   [83],
   [84],
   [85],
   [86],
   [87],
   [88],
   [89],
   [90],
   [91, 92],
   [93],
   [94]],
  'id': 53000

- It turns out that this `id` also contains to County FIPS codes


- So, we can use the these County FIPS codes to match the geographic data in `data.usa_10m` with the unemployment data in `data.unemployment`


- How? Let's start by making a map of the US counties:

In [25]:
alt.Chart(
    alt.topo_feature(us_url, 'counties')
).mark_geoshape(
    stroke='lightgray',
    strokeWidth=0.25
).project(
    type='albersUsa'
).properties(
    width=600,
    height=400
).configure_view(
    strokeWidth=0
)

alt.Chart(...)

- Next, we can use a *lookup transform* to retrieve the values of `rate` in `unemp_url` for each value of `id` that matches between `unemp_url` and `us_url`


- Then, we can encode the values of `rate` with `alt.Color()`

- The **lookup transform** extends the primary dataset given to a Chart object by looking up values from a secondary dataset


- The `transform_lookup()` method takes the following keyword arguments:
    - `lookup=...` specifies the variable in the Chart's primary dataset that we'll use as a key to look up values in the secondary dataset
    - `from=...` takes an `alt.LookupData` object which specifies the secondary dataset
    
    
- The `alt.LookupData()` method takes the following keyword arguments:
    - `data=...` specifies the secondary dataset
    - `key=...` specifies the variable in the secondary dataset to use as a key
        - The values of this variable should match values of the variable specified in `lookup=...` in `transform_lookup()`
    - `fields=...` specifies a list of variables in the secondary dataset to extract
    

- Let's put this all together:

In [26]:
# Solution
alt.Chart(
    alt.topo_feature(us_url, 'counties')
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(data=unemp_url, key='id', fields=['rate'])
).mark_geoshape(
    stroke='lightgray',
    strokeWidth=0.25
).encode(
    alt.Color('rate:Q'),
    alt.Tooltip('rate:Q')
).project(
    type='albersUsa'
).properties(
    width=600,
    height=400
).configure_view(
    strokeWidth=0
)

alt.Chart(...)

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Problems

**Problem 1.**
The `vega_datasets` package has a tabular dataset `data.zipcodes` of 5-digit zip codes in the United States. In particular, each row contains the `longitude` and `latitude` for the post office in each `zip_code`.

Create a map of the United States, with state boundaries, that represents each post office location with a 1-pixel square mark. You should end up with something that looks like this:

![](img/problem1.svg)

In [27]:
# Solution
from vega_datasets import data
zipcodes_url = data.zipcodes.url

base = alt.Chart(
    alt.topo_feature(us_url, 'states')
).mark_geoshape(
    fill='white', stroke='lightgray', strokeWidth=1
).project(
    type='albersUsa'
)

points = alt.Chart(zipcodes_url).mark_square(
    size=1
).encode(
    alt.Longitude('longitude:Q'),
    alt.Latitude('latitude:Q')
).project(
    type='albersUsa'
)

(base + points).properties(
    width=600,
    height=400
).configure_view(
    strokeWidth=0
)

alt.LayerChart(...)

**Problem 2.**
Is there any rhyme or reason to the allocation of zip codes?

Modify the map you created in Problem 1 by coloring the point for each zip code based on the *first digit* of the zip code.

*Hint.* Create a new variable containing the first digit of each zip code. In the Vega expression syntax, you can get the first character of a string-valued `variable` with `datum.variable[0]`, similar to what you would do in Python.

What do you see?

In [28]:
# Solution
from vega_datasets import data
zipcodes_url = data.zipcodes.url

base = alt.Chart(
    alt.topo_feature(us_url, 'states')
).mark_geoshape(
    fill='white', stroke='lightgray', strokeWidth=1
).project(
    type='albersUsa'
)

points = alt.Chart(
    zipcodes_url
).transform_calculate(
    digit='datum.zip_code[0]'
).mark_square(
    size=1
).encode(
    alt.Longitude('longitude:Q'),
    alt.Latitude('latitude:Q'),
    alt.Color('digit:N')
).project(
    type='albersUsa'
)

(base + points).properties(
    width=600,
    height=400
).configure_view(
    strokeWidth=0
)

alt.LayerChart(...)

**Problem 3.**
The `vega_datasets` package has a tabular dataset `data.population_engineers_hurricanes`.

1. Poke around the [vega-datasets website](https://github.com/vega/vega-datasets) and see if you can figure out what this dataset contains. See if you can find a description of the source of the data, as well as a preview of the data file itself.


2. The `'states'` object of the TopoJSON dataset `data.us_10m` that we used earlier in this lesson contains a list of `geographies`, each of which has an associated `id` value. These values turn out to be the [state FIPS codes](https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code). Is there something similar in `data.population_engineers_hurricanes`? Do some Google sleuthing to verify your hunch.


3. Use `data.us_10m` and `data.population_engineers_hurricanes` to create a chloropleth map of the United States that shows the population of each state with color. You should end up with something that looks like this:

![](img/problem3.svg)

In [29]:
# Solution
from vega_datasets import data
eng_url = data.population_engineers_hurricanes.url

alt.Chart(
    alt.topo_feature(us_url, 'states')
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(data=eng_url, key='id', fields=['population'])
).mark_geoshape(
    stroke='lightgray',
    strokeWidth=0.25
).encode(
    alt.Color('population:Q')
).project(
    type='albersUsa'
).properties(
    width=600,
    height=400
).configure_view(
    strokeWidth=0
)

alt.Chart(...)

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Notes and sources

- These lesson notes are based on these [Data Visualization Curriculum notebooks](https://github.com/uwdata/visualization-curriculum) by the University of Washington


- [Altair example gallery of map visualizations](https://altair-viz.github.io/gallery/index.html#maps)
